<a href="https://colab.research.google.com/github/elliotb01/linear-regression-machine-learning/blob/main/LinearRegression.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [16]:
houses = pd.DataFrame(pd.read_csv("/content/house_data.csv"))

# Data Inspection

houses.info()

,0
id,0.0
date,0.0
price,0.0
bedrooms,0.0
bathrooms,0.0
sqft_living,0.0
sqft_lot,0.0
floors,0.0
waterfront,0.0
view,0.0


In [17]:
# Data Cleaning

# Checking for any Null Values

houses.isnull().sum()*100/houses.shape[0]

,0
id,0.0
date,0.0
price,0.0
bedrooms,0.0
bathrooms,0.0
sqft_living,0.0
sqft_lot,0.0
floors,0.0
waterfront,0.0
view,0.0
